In [1]:
import tensorflow as tf


In [3]:
class nn(object):
    def __init__(self,input_size , hidden_size , output_size):
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.output_size = output_size
        
        #weight of input vector
        self.ui = tf.Variable(
            tf.random_normal([self.input_size,self.hidden_size],
                             mean=0,stddev=0.01,seed=1),name='ui')
        
        #weight of rest gate
        self.wr = tf.Variable(
            tf.random_normal([self.input_size,self.hidden_size],
                             mean=0,stddev=0.01,seed=1),name='wr')
        self.br = tf.Variable(
            tf.ones([self.hidden_size]),name='br')
        
        #weight of update gate
        self.wz = tf.Variable(
            tf.random_normal([self.input_size,self.hidden_size],
                             mean=0,stddev=0.01,seed=1),name='wz')
        self.bz = tf.Variable(
            tf.ones([self.hidden_size]),name='bz')

        #weight of hidden layer
        self.wh = tf.Variable(
            tf.random_normal([self.hidden_size,self.hidden_size],
                             mean=0,stddev=0.01,seed=1),name='wh')
        
        #weight of output layer
        self.vo = tf.Variable(
            tf.random_normal([self.hidden_size,output_size],
                             mean=0,stddev=0.01,seed=1),name='vo')
        self.bo = tf.Variable(
            tf.ones([self.output_size]),name='bo')
        
        #define shape of input matrix  
        
        self.inputs_matrix = tf.placeholder(
            dtype=tf.float32 ,shape=[None,None,self.input_size],name='inputs_matrix')
        
        #prepare input matrix to feed model 
        # it will make matrix as [m,t,n] --> [t,m,n]
        # t : number of time
        # n : length of input vector
        # m : number of sentences
        self.feed_marix = tf.transpose(
            self.inputs_matrix,perm=[1,0,2],name='feed_marix')
        
        #prepare initial state as matrix of zerose (batch_size,hidden_size)
        self.initial_state = tf.matmul(
            self.feed_marix[0],
            tf.zeros([self.input_size,self.hidden_size]),name='initial_state')
        
        self.a1 = tf.placeholder(tf.float32,name='a1')
        self.a1 = tf.placeholder(tf.float32, name='1')
        self.a2 = tf.placeholder(tf.float32, name='2')
        self.b1= tf.Variable(2.0,name="b")
    
    def ff(self):
        a = tf.multiply(self.a1,self.a2,name="abcd")
        return a
    def gru(self, previous_hidden_state, x):
        """gur cell that compute  curent state at time t using
           previous state at t-1 and input vector x
    
        Args:
            previous_hidden_state (matrix): previous_state at (t-1) .
            x (vector): input vector of char.

        Returns:
            current_hidden_state: current state

        Note:
            returned matrix size (batch_number , hidden_unit_size)
        """
        #compute update gate that update state_at_(t-1)
        z= tf.sigmoid(tf.matmul(x,self.wz)+ self.bz)
        #compute reset gate 
        r= tf.sigmoid(tf.matmul(x,self.wr)+ self.br)  
        #compute internal memory
        h_= tf.tanh(tf.matmul(x,self.ui) + tf.matmul(previous_hidden_state,self.wh)*r)
        #compute current state
        current_hidden_state = tf.multiply((1-z),h_) + tf.multiply(previous_hidden_state,z)
        return current_hidden_state  
        
    def get_states(self):
        """get all state of batch_number example it accumulate 
           all states of one sentence matrix like :
           S0 = initial_state + x0
           S1 = S0 + x1
           S2 = S1 + x2
                .
                .
           Sn = Sn-1 + xn
           
           and for batch_number
    
        Returns:
            all_stats: all state of m example

        Note:
            returned matrix size (number_time_steps ,batch_number , input_size )
        """
        # apply gru function on all matrix of all batch like
        #   state_0 =  gru(initial_state,feed_marix[0])
        #   state_1 =  gru(state_0,feed_matrix[1])
        all_stats = tf.scan(self.gru,self.feed_marix,self.initial_state,name="stats")
        return all_stats        
        
    def get_output(self,hidden_stat):
        """apply the hidden_state on RELU activation 
           function to compute output
        
        Args:
            hidden_stat (matrix): state at time t
        
        Returns:
            output: output matrix

        Note:
            returned matrix size (batch_number , output_size )
        """
        output = tf.nn.relu(tf.matmul(hidden_stat,self.vo)+self.bo,name='output')
        return output
        
n1 = nn(5,2,3)
a = n1.ff()
sess = tf.Session()
sess.run(tf.global_variables_initializer())
saver = tf.train.Saver()
sess.run(a,feed_dict={n1.a1:5,n1.a2:6})

30.0

In [4]:

saver.save(sess, 'Pramaters/my_test_model')

'Pramaters/my_test_model'

In [2]:
sess1 = tf.Session()    
saver = tf.train.import_meta_graph('Pramaters/my_test_model.meta')
saver.restore(sess1,tf.train.latest_checkpoint('Pramaters/'))
# op_to_restore = sess1.graph.get_tensor_by_name("abcd:0")
# print(sess1.run(op_to_restore))

graph = tf.get_default_graph()
a1 = graph.get_tensor_by_name('1:0')
a2 = graph.get_tensor_by_name('2:0')

aa = graph.get_tensor_by_name('abcd:0')

print(sess1.run(aa,feed_dict ={a1:15.0,a2:10.0}))




INFO:tensorflow:Restoring parameters from Pramaters/my_test_model
150.0


In [2]:
sess1 = tf.Session()    
saver = tf.train.import_meta_graph('Pramaters/Model_1_using_GRU.meta')
saver.restore(sess1,tf.train.latest_checkpoint('Pramaters/'))

INFO:tensorflow:Restoring parameters from Pramaters/Model_1_using_GRU


In [4]:
sess1.get_outputs()

AttributeError: 'Session' object has no attribute 'get_outputs'